In [47]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import time
import amt.method as md
import amt.data_loader as dl
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the data 

In [49]:
with open('../../parkinsons/parkinsons.pickle', 'rb') as f:
    X = pickle.load(f)
    y = pickle.load(f)
    miss_prop = pickle.load(f)
y = y-1
file_map = '../../parkinsons/parkinsons.map'
df_map = pd.read_csv(file_map, delimiter='\t', 
                     names=['chromosome', 'snp', 'start', 'end'])
n_sample, n_snp = X.shape
ind_snp = np.array(miss_prop<0.05, dtype=bool)
n_hypothesis = np.sum(ind_snp)

/home/martin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Compute the expected observations
Exp = np.zeros([8, n_snp], dtype=float)
for iy in range(2):
    for ix in range(4):
        Exp[iy*4+ix,:] = np.mean(y==iy) * np.mean(X==ix,axis=0)
Exp = Exp*n_sample
r_Exp = 1/Exp.clip(min=1e-6)*(Exp>0)
chi2_obs = md.compute_chi2(y, X, Exp, r_Exp)
data_gwas = {'X':X, 'y':y, 'Exp':Exp, 'r_Exp':r_Exp, 'chi2_obs':chi2_obs}

# Result analysis

In [50]:
snp_list = ['rs10501570', 'rs281357', 'rs2242330', 'rs1480597', 'rs6826751', 'rs4888984',
            'rs4862792', 'rs3775866', 'rs2235617', 'rs988421', 'rs7097094', 'rs999473',
            'rs1912373', 'rs1887279', 'rs2986574', 'rs11090762', 'rs6125829', 'rs7796855',
            'rs355477', 'rs3010040', 'rs2296713', 'rs355461', 'rs355506', 'rs355464',
            'rs1497430', 'rs11946612']
# result_file = '/home/martin/adapative_MC_test/results/GWAS/GWAS_amt_alpha_0.005_rep1.pickle'
# result_file = '/home/martin/adapative_MC_test/results/GWAS/GWAS_amt_alpha_0.05_rep0.pickle'
result_file = '../../results/GWAS/GWAS_amt_alpha_0.1_rep0.pickle'
with open(result_file, 'rb')as f:
    res_AMT = pickle.load(f)
h_amt = res_AMT['h_amt']
p_amt_ub = res_AMT['p_hat_ub']
p_amt_lb = res_AMT['p_hat_lb']
p_amt = res_AMT['p_hat']
print(np.sum(h_amt))
# print(np.arange(h_amt.shape[0])[h_amt])

304


In [51]:
# df_map_c4 = df_map.loc[df_map['chromosome']==4]
n_overlap = 0
temp_df_map = df_map.loc[ind_snp]
for snp in snp_list:
    temp = np.array(temp_df_map['snp']==snp, dtype=bool)
    if np.sum(temp)>0:
        print('######################')
        print(snp)
        print(temp_df_map.loc[temp])
        print('decision', h_amt[temp])
        print('miss_prop=%0.4f, p_amc='%(miss_prop[ind_snp][temp]),
                                         p_amt_lb[temp], p_amt_ub[temp], p_amt[temp])
        if h_amt[temp][0]:
            n_overlap+=1
        print('')

######################
rs10501570
       chromosome         snp  start       end
262094         11  rs10501570      0  84095494
decision [ True]
miss_prop=0.0019, p_amc= [-9.59697468e-06] [6.93645165e-05] [3.8033274e-06]

######################
rs281357
       chromosome       snp  start       end
343235         17  rs281357      0  19683106
decision [ True]
miss_prop=0.0000, p_amc= [-6.5849564e-06] [7.24366922e-05] [9.21916018e-06]

######################
rs2242330
       chromosome        snp  start       end
100847          4  rs2242330      0  68276015
decision [ True]
miss_prop=0.0000, p_amc= [-6.5849564e-06] [7.24366922e-05] [9.21916018e-06]

######################
rs1480597
       chromosome        snp  start       end
237247         10  rs1480597      0  44481115
decision [ True]
miss_prop=0.0223, p_amc= [-2.11240548e-06] [7.17599404e-05] [1.52353239e-05]

######################
rs6826751
       chromosome        snp  start       end
100837          4  rs6826751      0  6826262